# International Finance, 
Thomas de Portzamparc - 1/12/2025 

# Imports

In [27]:
import numpy as np 
import pandas as pd
import yfinance as yf
from scipy.stats import skew, kurtosis


import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# Data Loading

In [73]:
dict_forward = pd.read_excel("fwd_rates.xlsx", header = 0, skiprows = [2], sheet_name = None, index_col = 0)
df_forward = pd.concat(dict_forward.values(), axis = 1)
df_forward = df_forward[1:]
dict_spot = pd.read_excel("spot_rates.xls", header = [0, 1], sheet_name = None, index_col = 0)
df_spot = pd.concat(dict_spot.values(), axis = 1)

/opt/anaconda3/lib/python3.12/site-packages/openpyxl/worksheet/_read_only.py:85: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


In [67]:
# Here we will get the dollar exchange rate and remove the other unused columns to manipulate the dataframe quicker
usd_columns_spot = [col for col in df_spot if " US" in col[0] or "US " in col [0]] 
usd_columns_fwd = [col for col in df_forward if " US" in col]
usd_columns_spot, usd_columns_fwd # A lot of XUSD spot columns are missing, we may need to retreive them buy using other currency pairs

([('US $ TO EURO (WMR) - BID SPOT', 'USEURSP(EB)'),
  ('US $ TO EURO (WMR) - EXCHANGE RATE', 'USEURSP(ER)'),
  ('US $ TO EURO (WMR) - SPOT OFFERED', 'USEURSP(EO)'),
  ('US $ TO UK £ (WMR) - BID SPOT', 'USDOLLR(EB)'),
  ('US $ TO UK £ (WMR) - EXCHANGE RATE', 'USDOLLR(ER)'),
  ('US $ TO UK £ (WMR) - SPOT OFFERED', 'USDOLLR(EO)')],
 ['BRAZILIAN REAL TO US $ 1M FWD OFF TR - BID SPOT',
  'BRAZILIAN REAL TO US $ 1M FWD OFF TR - EXCHANGE RATE',
  'BRAZILIAN REAL TO US $ 1M FWD OFF TR - SPOT OFFERED',
  'BRAZILIAN REAL TO US $ 1W NDF (WMR) - BID SPOT',
  'BRAZILIAN REAL TO US $ 1W NDF (WMR) - EXCHANGE RATE',
  'BRAZILIAN REAL TO US $ 1W NDF (WMR) - SPOT OFFERED',
  'CANADIAN $ TO US $ 1M FWD (BBI) - BID SPOT',
  'CANADIAN $ TO US $ 1M FWD (BBI) - EXCHANGE RATE',
  'CANADIAN $ TO US $ 1M FWD (BBI) - SPOT OFFERED',
  'CANADIAN $ TO US $ 1W FWD (TR) - BID SPOT',
  'CANADIAN $ TO US $ 1W FWD (TR) - EXCHANGE RATE',
  'CANADIAN $ TO US $ 1W FWD (TR) - SPOT OFFERED',
  'CROATIAN KUNA TO US $ 1M FWD (

# Data Pre - Processing 
Here we will run some pre - treatment prior to executing strategies for both spot and forward dataframes
---

## Spot dataframe

### Computing of the XUSD spot

In [68]:
import pandas as pd
import numpy as np

# =========================================================
# 1) Construire meta (Cur1 / Cur2 / Type)
# =========================================================

records = []

for (title, code) in df_spot.columns:
    left, right = title.split("TO")
    cur1 = left.strip().split()[0].upper()
    cur2 = right.strip().split()[0].upper()
    
    raw_nature = title.split("-")[-1].strip().upper()

    if "BID" in raw_nature:
        price_type = "BID"
    elif "OFFER" in raw_nature:
        price_type = "OFFER"
    elif "EXCHANGE" in raw_nature:
        price_type = "ER"
    else:
        price_type = "OTHER"

    records.append({
        "Title": title,
        "Code": code,
        "Cur1": cur1,
        "Cur2": cur2,
        "RawNature": raw_nature,
        "Type": price_type
    })

meta = pd.DataFrame(records, index=df_spot.columns)

# Normalisation MultiIndex
df_spot.columns = pd.MultiIndex.from_tuples([(str(a), str(b)) for a, b in df_spot.columns])
meta.index       = pd.MultiIndex.from_tuples([(str(a), str(b)) for a, b in meta.index])

# =========================================================
# 2) Fonctions utilitaires
# =========================================================

def get_leg(cur1, cur2, typ):
    """Renvoie la série correspondant à la paire cur1->cur2 au type BID/OFFER/ER."""
    mask = (meta["Cur1"] == cur1) & (meta["Cur2"] == cur2) & (meta["Type"] == typ)
    idx = meta.index[mask]
    if len(idx) == 0:
        return None
    return df_spot[idx[0]]

def invert_bid_ask(bid, ask):
    """Inverse une cotation A→B pour obtenir B→A."""
    return 1/ask, 1/bid

# =========================================================
# 3) On construit XUSD = USD per X (intermédiaire)
# =========================================================

xusd = pd.DataFrame(index=df_spot.index)

currencies = set(meta["Cur1"].unique()) | set(meta["Cur2"].unique())
currencies.discard("US")

# --- Traitement direct pour UK & EURO (US→X disponible)
for tgt in ["UK", "EURO"]:
    bid_US_X   = get_leg("US", tgt, "BID")
    offer_US_X = get_leg("US", tgt, "OFFER")
    mid_US_X   = get_leg("US", tgt, "ER")

    if bid_US_X is not None and offer_US_X is not None:
        bid_X_US, offer_X_US = invert_bid_ask(bid_US_X, offer_US_X)
        mid_X_US = 1/mid_US_X if mid_US_X is not None else (bid_X_US + offer_X_US)/2

        xusd[f"{tgt}_BID"]   = bid_X_US
        xusd[f"{tgt}_OFFER"] = offer_X_US
        xusd[f"{tgt}_ER"]    = mid_X_US

# --- Cross avec pivots pour les autres
pivots = ["UK", "EURO"]

for cur in sorted(currencies):
    if cur in ["UK", "EURO"]:
        continue

    for pivot in pivots:

        bid_X_P   = get_leg(cur,  pivot, "BID")
        offer_X_P = get_leg(cur,  pivot, "OFFER")
        mid_X_P   = get_leg(cur,  pivot, "ER")

        bid_US_P   = get_leg("US", pivot, "BID")
        offer_US_P = get_leg("US", pivot, "OFFER")
        mid_US_P   = get_leg("US", pivot, "ER")

        if bid_X_P is None or offer_X_P is None or bid_US_P is None or offer_US_P is None:
            continue

        # USD per X
        bid_X_US   = bid_X_P   / offer_US_P
        offer_X_US = offer_X_P / bid_US_P
        mid_X_US   = (mid_X_P/mid_US_P) if (mid_X_P is not None and mid_US_P is not None) else (bid_X_US + offer_X_US)/2

        xusd[f"{cur}_BID"]   = bid_X_US
        xusd[f"{cur}_OFFER"] = offer_X_US
        xusd[f"{cur}_ER"]    = mid_X_US

        break

# =========================================================
# 4) Conversion XUSD -> USDX (ce que tu veux réellement)
#    EURUSD = euros pour un USD
# =========================================================

usd_df = pd.DataFrame(index=xusd.index)

for col in xusd.columns:
    cur, typ = col.split("_")      # ex: EURO, BID -> EURO_BID
    
    if typ == "BID":
        newcol = f"{cur}_OFFER"
        usd_df[newcol] = 1 / xusd[col]

    elif typ == "OFFER":
        newcol = f"{cur}_BID"
        usd_df[newcol] = 1 / xusd[col]

    else:  # ER
        newcol = f"{cur}_ER"
        usd_df[newcol] = 1 / xusd[col]

# Tri propre
usd_df = usd_df.sort_index(axis=1)

usd_df.columns


Index(['AUSTRALIAN_BID', 'AUSTRALIAN_ER', 'AUSTRALIAN_OFFER', 'BRAZILIAN_BID',
       'BRAZILIAN_ER', 'BRAZILIAN_OFFER', 'BULGARIAN_BID', 'BULGARIAN_ER',
       'BULGARIAN_OFFER', 'CANADIAN_BID', 'CANADIAN_ER', 'CANADIAN_OFFER',
       'CHILEAN_BID', 'CHILEAN_ER', 'CHILEAN_OFFER', 'CROATIAN_BID',
       'CROATIAN_ER', 'CROATIAN_OFFER', 'CZECH_BID', 'CZECH_ER', 'CZECH_OFFER',
       'EURO_BID', 'EURO_ER', 'EURO_OFFER', 'HUNGARIAN_BID', 'HUNGARIAN_ER',
       'HUNGARIAN_OFFER', 'INDIAN_BID', 'INDIAN_ER', 'INDIAN_OFFER',
       'INDONESIAN_BID', 'INDONESIAN_ER', 'INDONESIAN_OFFER', 'ISRAELI_BID',
       'ISRAELI_ER', 'ISRAELI_OFFER', 'JAPANESE_BID', 'JAPANESE_ER',
       'JAPANESE_OFFER', 'MEXICAN_BID', 'MEXICAN_ER', 'MEXICAN_OFFER',
       'NEW_BID', 'NEW_ER', 'NEW_OFFER', 'NORWEGIAN_BID', 'NORWEGIAN_ER',
       'NORWEGIAN_OFFER', 'PHILIPPINE_BID', 'PHILIPPINE_ER',
       'PHILIPPINE_OFFER', 'POLISH_BID', 'POLISH_ER', 'POLISH_OFFER',
       'RUSSIAN_BID', 'RUSSIAN_ER', 'RUSSIAN_OFFER', '

### Coherence of the spot obtained 
The first thing to do here is to verify the coherence of our computing, to do this we have several ressources, chatgpt and other AI tool may help us quickly review our code but to check the coherence of our data we can look at some spots on Yfinance or do it empirically as we've done below 

In [16]:
# Dictionnary containing the index of the comparison dataframe and the yfinance ticker to extract market data
fx_map = {
    "UK": "GBPUSD=X",
    "EURO": "EURUSD=X",
    "PHILIPPINE": "PHPUSD=X",
    "CANADIAN": "CADUSD=X",
    "NORWEGIAN": "NOKUSD=X",
    "NEW": "NZDUSD=X",
    "CZECH": "CZKUSD=X",
    "HUNGARIAN": "HUFUSD=X",
    "POLISH": "PLNUSD=X",
    "SINGAPORE": "SGDUSD=X",
    "RUSSIAN": "RUBUSD=X",
    "INDIAN": "INRUSD=X",
    "SOUTH": "ZARUSD=X",
    "INDONESIAN": "IDRUSD=X",
    "BULGARIAN": "BGNUSD=X",
    "ISRAELI": "ILSUSD=X",
    "JAPANESE": "JPYUSD=X",
    "BRAZILIAN": "BRLUSD=X",
    "SWEDISH": "SEKUSD=X",
    "THAI": "THBUSD=X",
    "AUSTRALIAN": "AUDUSD=X",
    "SWISS": "CHFUSD=X",
    "MEXICAN": "MXNUSD=X",
    "CHILEAN": "CLPUSD=X",
}

target_date = "2024-10-23"
results = {}

for name, ticker in fx_map.items():
    try:
        data = yf.download(
            ticker,
            start="2024-10-23",
            end="2024-10-24",
            progress=False,
            auto_adjust=False
        )
        
        # If no data → record NaN
        if data.empty:
            results[name+"_ER"] = float("nan")
            continue
        
        # Look for the exact date
        date_match = data.loc[data.index.strftime("%Y-%m-%d") == target_date]
        
        if len(date_match) == 0:
            results[name+"_ER"] = float("nan")
        else:
            results[name+"_ER"] = date_match["Close"].iloc[0]
    
    except Exception:
        results[name] = float("nan")

clean_results = {k: float(v.iloc[0])for k, v in results.items()}
df_check = pd.DataFrame.from_dict(clean_results,orient="index", columns=["USD per X"]) # dataframe 


# DATA comparison 
row_model = usd_df.loc[target_date]
row_model.name = "USD_per_X_professor_data"
df_model = row_model.to_frame(name="USD_per_X_model")
comparison = df_model.join(df_check, how="inner")
comparison["abs_diff"] = comparison["USD_per_X_model"] - comparison["USD per X"]
comparison["rel_diff(%)"] = comparison["abs_diff"] / comparison["USD per X"] * 100

print(comparison.sort_values("rel_diff(%)"))


               USD_per_X_model  USD per X      abs_diff  rel_diff(%)
SOUTH_ER              0.056046   0.057065 -1.019226e-03    -1.786071
JAPANESE_ER           0.006534   0.006616 -8.244308e-05    -1.246028
HUNGARIAN_ER          0.002671   0.002698 -2.751943e-05    -1.019843
NORWEGIAN_ER          0.090737   0.091561 -8.240841e-04    -0.900040
POLISH_ER             0.247830   0.249814 -1.984443e-03    -0.794367
AUSTRALIAN_ER         0.663549   0.668600 -5.050777e-03    -0.755426
ISRAELI_ER            0.263613   0.265449 -1.835817e-03    -0.691589
RUSSIAN_ER            0.010373   0.010445 -7.121755e-05    -0.681857
BRAZILIAN_ER          0.174618   0.175769 -1.150380e-03    -0.654485
NEW_ER                0.600645   0.604580 -3.935451e-03    -0.650939
SWEDISH_ER            0.094266   0.094862 -5.964737e-04    -0.628779
BULGARIAN_ER          0.551086   0.554227 -3.140510e-03    -0.566647
SINGAPORE_ER          0.755617   0.759803 -4.186197e-03    -0.550958
CZECH_ER              0.042665   0

In [21]:
# Once the verification is done, we can pursue our calculus without worrying about wether our currency pairs are quoted in the wrong direction
# First and foremost we will thus start by computing some log returns -> we pick this because it has the nice property that the returns are additive and because the 
# subject encourage us to go this way 

## Forward dataframe

In [77]:
col_1W = [col for col in usd_columns_fwd if "1W" in col]
col_1M = [col for col in usd_columns_fwd if "1M" in col]
col_1M

['BRAZILIAN REAL TO US $ 1M FWD OFF TR - BID SPOT',
 'BRAZILIAN REAL TO US $ 1M FWD OFF TR - EXCHANGE RATE',
 'BRAZILIAN REAL TO US $ 1M FWD OFF TR - SPOT OFFERED',
 'CANADIAN $ TO US $ 1M FWD (BBI) - BID SPOT',
 'CANADIAN $ TO US $ 1M FWD (BBI) - EXCHANGE RATE',
 'CANADIAN $ TO US $ 1M FWD (BBI) - SPOT OFFERED',
 'CROATIAN KUNA TO US $ 1M FWD (WMR) - BID SPOT',
 'CROATIAN KUNA TO US $ 1M FWD (WMR) - EXCHANGE RATE',
 'CROATIAN KUNA TO US $ 1M FWD (WMR) - SPOT OFFERED',
 'CZECH KORUNA TO US $ 1M FWD (TR) - BID SPOT',
 'CZECH KORUNA TO US $ 1M FWD (TR) - EXCHANGE RATE',
 'CZECH KORUNA TO US $ 1M FWD (TR) - SPOT OFFERED',
 'HUNGARIAN HUF TO US $ 1M FWD (WMR) - BID SPOT',
 'HUNGARIAN HUF TO US $ 1M FWD (WMR) - EXCHANGE RATE',
 'HUNGARIAN HUF TO US $ 1M FWD (WMR) - SPOT OFFERED',
 'INDIAN RUPEE TO US $ 1M FWD (WMR) - BID SPOT',
 'INDIAN RUPEE TO US $ 1M FWD (WMR) - EXCHANGE RATE',
 'INDIAN RUPEE TO US $ 1M FWD (WMR) - SPOT OFFERED',
 'INDONESIAN RUPIAH TO US $ 1M FWD - BID SPOT',
 'INDONESI

In [90]:
import pandas as pd
import numpy as np
import re

# ----------------------------------------------------------
# 0) Fix non-1D columns (critical)
# ----------------------------------------------------------
def force_series(df):
    df2 = df.copy()
    for c in df2.columns:
        col = df2[c]

        # if a cell contains a list or array → flatten column
        if any(isinstance(x, (list, np.ndarray)) for x in col):
            df2[c] = col.apply(lambda x: x[0] if isinstance(x, (list, np.ndarray)) else x)

    return df2


df_forward = force_series(df_forward)


# ----------------------------------------------------------
# 1) Currency normalization
# ----------------------------------------------------------
CURRENCY_MAP = {
    "BRL": "BRAZILIAN",
    "BRAZILIAN": "BRAZILIAN",
    "CAD": "CANADIAN",
    "CANADIAN": "CANADIAN",
    "HRK": "CROATIAN",
    "CROATIAN": "CROATIAN",
    "CZK": "CZECH",
    "CZECH": "CZECH",
    "HUF": "HUNGARIAN",
    "HUNGARIAN": "HUNGARIAN",
    "INR": "INDIAN",
    "INDIAN": "INDIAN",
    "IDR": "INDONESIAN",
    "INDONESIAN": "INDONESIAN",
    "ILS": "ISRAELI",
    "ISRAELI": "ISRAELI",
    "JPY": "JAPANESE",
    "JAPANESE": "JAPANESE",
    "MXN": "MEXICAN",
    "MEXICAN": "MEXICAN",
    "NOK": "NORWEGIAN",
    "NORWEGIAN": "NORWEGIAN",
    "PHP": "PHILIPPINE",
    "PHILIPPINE": "PHILIPPINE",
    "PLN": "POLISH",
    "POLISH": "POLISH",
    "RUB": "RUSSIAN",
    "RUSSIAN": "RUSSIAN",
    "SGD": "SINGAPORE",
    "SINGAPORE": "SINGAPORE",
    "ZAR": "SOUTH",
    "SOUTH": "SOUTH",
    "SEK": "SWEDISH",
    "SWEDISH": "SWEDISH",
    "CHF": "SWISS",
    "SWISS": "SWISS",
    "THB": "THAI",
    "THAI": "THAI",
}


def clean_cur(raw):
    raw = re.sub(r"[^A-Z]", "", raw.upper())
    return CURRENCY_MAP.get(raw, raw)


# ----------------------------------------------------------
# 2) Clean forward columns into forward_1M / forward_1W
# ----------------------------------------------------------
def build_forward_clean(df):

    data_1M = {}
    data_1W = {}

    for col in df.columns:

        title = col.upper()

        # extract currency before "TO"
        if "TO" not in title:
            continue

        left, _ = title.split("TO", 1)
        cur_raw = left.strip().split()[0]
        cur = clean_cur(cur_raw)

        if cur in ["US", "USD"]:
            continue

        # tenor
        if "1M" in title:
            tenor = "1M"
        elif "1W" in title:
            tenor = "1W"
        else:
            continue

        # type
        if "BID" in title:
            typ = "BID"
        elif "OFFER" in title:
            typ = "OFFER"
        elif "EXCHANGE RATE" in title:
            typ = "ER"
        else:
            continue

        new_name = f"{cur}_{typ}"

        if tenor == "1M":
            data_1M[new_name] = df[col]
        else:
            data_1W[new_name] = df[col]

    df_1M = pd.DataFrame(data_1M, index=df.index).sort_index(axis=1)
    df_1W = pd.DataFrame(data_1W, index=df.index).sort_index(axis=1)

    return df_1M, df_1W


forward_1M, forward_1W = build_forward_clean(df_forward)

print("FORWARD 1M:", forward_1M.columns)
print("FORWARD 1W:", forward_1W.columns)


ValueError: 2

In [89]:
forward_1M

""
1998-11-01 00:00:00
1998-12-01 00:00:00
1999-01-01 00:00:00
1999-02-01 00:00:00
1999-03-01 00:00:00
...
2024-06-01 00:00:00
2024-07-01 00:00:00
2024-08-01 00:00:00
2024-09-01 00:00:00


In [75]:
import pandas as pd
import re

def normalize_forward_columns(df_forward):
    new_cols = []

    for title in df_forward.columns:
        t = str(title).upper()

        # --- Extract currency (first token before "TO")
        if "TO" in t:
            cur1 = t.split("TO", 1)[0].strip().split()[0]
        else:
            cur1 = t.strip().split()[0]

        # Standardize currency names (match usd_df conventions)
        mapping = {
            "BRAZILIAN": "BRAZILIAN",
            "CANADIAN": "CANADIAN",
            "CROATIAN": "CROATIAN",
            "CZECH": "CZECH",
            "HUNGARIAN": "HUNGARIAN",
            "HUF": "HUNGARIAN",
            "INDIAN": "INDIAN",
            "INDONESIAN": "INDONESIAN",
            "ISRAELI": "ISRAELI",
            "JAPANESE": "JAPANESE",
            "MEXICAN": "MEXICAN",
            "NORWEGIAN": "NORWEGIAN",
            "PHILIPPINE": "PHILIPPINE",
            "POLISH": "POLISH",
            "RUSSIAN": "RUSSIAN",
            "SINGAPORE": "SINGAPORE",
            "SOUTH": "SOUTH",
            "SWEDISH": "SWEDISH",
            "SWISS": "SWISS",
            "THAI": "THAI",
            "EURO": "EURO",
            "UK": "UK",
        }

        cur_std = mapping.get(cur1, cur1)

        # --- Extract type: BID / OFFER / ER
        if "BID" in t:
            typ = "BID"
        elif "OFFER" in t or "OFFERED" in t:
            typ = "OFFER"
        elif "EXCHANGE RATE" in t:
            typ = "ER"
        else:
            typ = "OTHER"

        new_cols.append(f"{cur_std}_{typ}")

    # Apply renaming
    df_forward_renamed = df_forward.copy()
    df_forward_renamed.columns = new_cols

    return df_forward_renamed
forward_clean = normalize_forward_columns(df_forward)
forward_clean.head().columns


Index(['US_BID', 'US_ER', 'US_OFFER', 'US_BID', 'US_ER', 'US_OFFER',
       'BRAZILIAN_BID', 'BRAZILIAN_ER', 'BRAZILIAN_OFFER', 'BRAZILIAN_BID',
       ...
       'THB_OFFER', 'THB_BID', 'THB_ER', 'THB_OFFER', 'US_BID', 'US_ER',
       'US_OFFER', 'US_BID', 'US_ER', 'US_OFFER'],
      dtype='object', length=301)

# Trading strategies 

## Momentum strategy

In [25]:
# =====================================================
# 1. Split the dataframe into BID / ER (spot mid) / OFFER
# =====================================================

def split_bid_offer(df):
    bid_cols   = [c for c in df.columns if c.endswith("_BID")]
    er_cols    = [c for c in df.columns if c.endswith("_ER")]    # mid spot prices
    offer_cols = [c for c in df.columns if c.endswith("_OFFER")]

    BID   = df[bid_cols].copy()
    ER    = df[er_cols].copy()
    OFFER = df[offer_cols].copy()

    BID.columns   = [c.replace("_BID", "") for c in bid_cols]
    ER.columns    = [c.replace("_ER", "") for c in er_cols]
    OFFER.columns = [c.replace("_OFFER", "") for c in offer_cols]

    return BID, ER, OFFER


# =====================================================
# 2. Momentum strategy consistent with assignment instructions
#    - Signal based on 4-day cumulative log returns (mid prices)
#    - Positions held for one month (22 days)
#    - Execution priced with BID/OFFER
# =====================================================

def momentum_strategy(df, lookback=4, hold_period=22, pct=0.3):
    
    BID, ER, OFFER = split_bid_offer(df)

    # -----------------------------
    # 2a. Daily log spot returns
    # -----------------------------
    log_ret_spot = np.log(ER).diff()

    # cumulative return over lookback window
    mom_signal = log_ret_spot.rolling(lookback).sum()

    dates   = ER.index
    assets  = ER.columns
    signals = pd.DataFrame(0, index=dates, columns=assets)

    # rebalance dates: every 22 days after lookback
    rebalance_dates = dates[lookback::hold_period]

    # -----------------------------
    # Assign long/short portfolios
    # -----------------------------
    for t in rebalance_dates:
        today = mom_signal.loc[t].dropna()
        if today.empty:
            continue

        n = len(today)
        k = int(np.floor(n * pct))

        winners = today.nlargest(k).index
        losers  = today.nsmallest(k).index

        signals.loc[t, winners] = 1
        signals.loc[t, losers]  = -1

    # hold the positions for one month
    signals = signals.replace(0, np.nan).ffill().fillna(0)

    # -----------------------------
    # Execution with bid–ask structure
    # -----------------------------
    long_ret  = np.log(BID)   - np.log(OFFER.shift(1))
    short_ret = np.log(OFFER) - np.log(BID.shift(1))

    executed = (
        (signals.shift(1) == 1)  * long_ret +
        (signals.shift(1) == -1) * short_ret
    )

    strat_ret = executed.mean(axis=1)

    return strat_ret, signals, mom_signal, log_ret_spot


# =====================================================
# 3. Evaluation function (annualized statistics)
# =====================================================

def evaluate_strategy(r):

    perf = (1 + r).cumprod()

    ann_ret = (1 + r.mean())**252 - 1
    ann_vol = r.std() * np.sqrt(252)
    sharpe  = ann_ret / ann_vol if ann_vol != 0 else np.nan

    running_max = perf.cummax()
    max_dd = ((perf - running_max) / running_max).min()

    return pd.Series({
        "Annualized Return": ann_ret,
        "Annualized Volatility": ann_vol,
        "Sharpe Ratio": sharpe,
        "Max Drawdown": max_dd,
        "Skewness (daily)": skew(r.dropna()),
        "Kurtosis (daily)": kurtosis(r.dropna(), fisher=False)
    })


# =====================================================
# 4. Run strategy
# =====================================================

strat_ret, signals, momentum, log_ret_spot = momentum_strategy(
    usd_df,
    lookback=4,
    hold_period=22,
    pct=0.3
)

stats = evaluate_strategy(strat_ret)
print(stats)


Annualized Return       -0.017119
Annualized Volatility    0.067577
Sharpe Ratio            -0.253324
Max Drawdown            -0.488526
Skewness (daily)        -0.123728
Kurtosis (daily)         7.376205
dtype: float64


In [26]:
param_grid = [
    {"lookback": 4, "hold": 22, "pct": 0.2},
    {"lookback": 12, "hold": 22, "pct": 0.2},
    {"lookback": 4, "hold": 10, "pct": 0.2},
    {"lookback": 12, "hold": 10, "pct": 0.2},
    {"lookback": 4, "hold": 22, "pct": 0.4},
    {"lookback": 12, "hold": 22, "pct": 0.4},
    {"lookback": 4, "hold": 10, "pct": 0.4},
    {"lookback": 12, "hold": 10, "pct": 0.2},
]
subsamples = {
    "1999-2007": ("1999-01-01", "2007-12-31"), # (pre-crisis)
    "2008-2012": ("2008-01-01", "2012-12-31"), # (Financial Crisis + Euro Crisis)
    "2013-2019": ("2013-01-01", "2019-12-31"), # (QE, Low Volatility)
    "2020-2024": ("2020-01-01", "2024-12-31"), # (COVID, Inflation, QT)
}


results_sub = {}

for sub_name, (start, end) in subsamples.items():
    df_sub = usd_df.loc[start:end]

    if len(df_sub) < 200:
        continue

    for p in param_grid:
        lb = p["lookback"]
        hd = p["hold"]
        pct = p["pct"]

        strat_ret, signals, mom,_ = momentum_strategy(
            df_sub, 
            lookback=lb,
            hold_period=hd,
            pct=pct
        )

        stats = evaluate_strategy(strat_ret)

        key = f"{sub_name} | LB={lb}, HP={hd}, PCT={pct}"
        results_sub[key] = stats

subsample_comparison = pd.DataFrame(results_sub).T
subsample_comparison


,Annualized Return,Annualized Volatility,Sharpe Ratio,Max Drawdown,Skewness (daily),Kurtosis (daily)
"1999-2007 | LB=4, HP=22, PCT=0.2",0.018728,0.051404,0.364336,-0.169903,-0.122593,4.218853
"1999-2007 | LB=12, HP=22, PCT=0.2",0.009646,0.051310,0.187988,-0.251964,-0.068492,4.209252
"1999-2007 | LB=4, HP=10, PCT=0.2",0.029216,0.051575,0.566485,-0.156491,-0.072301,4.183647
"1999-2007 | LB=12, HP=10, PCT=0.2",0.001179,0.051528,0.022889,-0.280843,-0.057716,4.156313
"1999-2007 | LB=4, HP=22, PCT=0.4",0.018506,0.051961,0.356149,-0.206534,-0.078269,4.090814
"1999-2007 | LB=12, HP=22, PCT=0.4",0.013586,0.051559,0.263513,-0.195926,-0.060692,4.173402
"1999-2007 | LB=4, HP=10, PCT=0.4",0.021204,0.051709,0.410058,-0.165224,-0.069525,4.146124
"2008-2012 | LB=4, HP=22, PCT=0.2",-0.029759,0.093202,-0.319299,-0.269846,0.046087,5.809304
"2008-2012 | LB=12, HP=22, PCT=0.2",-0.022311,0.090079,-0.247687,-0.259974,0.064796,5.568416
"2008-2012 | LB=4, HP=10, PCT=0.2",-0.026070,0.093597,-0.278532,-0.265670,0.068300,5.868482


In [40]:
import pandas as pd
import numpy as np

# =====================================================
# 0. Use same bid/offer split as momentum strategy
# =====================================================

def split_bid_offer(df):
    bid_cols   = [c for c in df.columns if c.endswith("_BID")]
    er_cols    = [c for c in df.columns if c.endswith("_ER")]
    offer_cols = [c for c in df.columns if c.endswith("_OFFER")]

    BID   = df[bid_cols].copy()
    ER    = df[er_cols].copy()
    OFFER = df[offer_cols].copy()

    BID.columns   = [c.replace("_BID", "") for c in bid_cols]
    ER.columns    = [c.replace("_ER", "") for c in er_cols]
    OFFER.columns = [c.replace("_OFFER", "") for c in offer_cols]

    return BID, ER, OFFER


# =====================================================
# 1. PPP Strategy (long undervalued FX, short overvalued FX)
# =====================================================

def ppp_strategy(df_spot, df_forward, hold_period=22, pct=0.3):
    """
    df_spot    = usd_df (with BID / ER / OFFER)
    df_forward = forward_1M or forward_1W
    """

    BID, ER, OFFER = split_bid_offer(df_spot)

    # -----------------------------
    # PPP VALUE SIGNAL
    # -----------------------------
    # Align forward on spot index and columns
    F = df_forward.reindex(index=ER.index, columns=ER.columns)

    # PPP mispricing: positive → overvalued → short
    value = F / ER - 1
    if isnotnone(value):
        print(value)
    dates = ER.index
    assets = ER.columns

    signals = pd.DataFrame(0, index=dates, columns=assets)

    rebalance_dates = dates[::hold_period]

    for t in rebalance_dates:

        if t not in value.index:
            continue

        today_val = value.loc[t].dropna()
        if today_val.empty:
            continue

        n = len(today_val)
        k = int(np.floor(n * pct))

        # undervalued (negative) → long
        underval = today_val.nsmallest(k).index
        
        # overvalued (positive) → short
        overval  = today_val.nlargest(k).index

        signals.loc[t, underval] = 1
        signals.loc[t, overval]  = -1

    signals = signals.replace(0, np.nan).ffill().fillna(0)

    # -----------------------------
    # EXECUTION WITH BID/OFFER
    # -----------------------------
    long_ret  = np.log(BID)   - np.log(OFFER.shift(1))
    short_ret = np.log(OFFER) - np.log(BID.shift(1))

    executed = (
        (signals.shift(1) == 1)  * long_ret +
        (signals.shift(1) == -1) * short_ret
    )

    strat_ret = executed.mean(axis=1)

    return strat_ret, signals, value

# PPP 1-Month strategy
ppp_ret_1M, ppp_sig_1M, value_1M = ppp_strategy(
    usd_df,
    forward_1M,
    hold_period=4,
    pct=0.3
)

# PPP 1-Week strategy
ppp_ret_1W, ppp_sig_1W, value_1W = ppp_strategy(
    usd_df,
    forward_1W,
    hold_period=4,
    pct=0.3
)

stats_ppp_1M = evaluate_strategy(ppp_ret_1M)
print(stats)


stats_ppp_1W = evaluate_strategy(ppp_ret_1W)
print(stats)

NameError: name 'isnotnone' is not defined

In [43]:
print(forward_1M.equals(forward_1W))
print(forward_1M.dropna(how="all").shape)
print(forward_1W.dropna(how="all").shape)
forward_1W.columns

False
(321, 105)
(321, 72)


Index(['BRAZILIAN_BID', 'BRAZILIAN_OFFER', 'BRAZILIAN_ER', 'CAD_BID',
       'CAD_OFFER', 'CAD_ER', 'CANADIAN_BID', 'CANADIAN_OFFER', 'CANADIAN_ER',
       'CROATIAN_BID', 'CROATIAN_OFFER', 'CROATIAN_ER', 'CZECH_BID',
       'CZECH_OFFER', 'CZECH_ER', 'CZK_BID', 'CZK_OFFER', 'CZK_ER',
       'HUNGARIAN_BID', 'HUNGARIAN_OFFER', 'HUNGARIAN_ER', 'INDIAN_BID',
       'INDIAN_OFFER', 'INDIAN_ER', 'INDONESIAN_BID', 'INDONESIAN_OFFER',
       'INDONESIAN_ER', 'ISRAELI_BID', 'ISRAELI_OFFER', 'ISRAELI_ER',
       'JAPANESE_BID', 'JAPANESE_OFFER', 'JAPANESE_ER', 'JPY_BID', 'JPY_OFFER',
       'JPY_ER', 'MEXICAN_BID', 'MEXICAN_OFFER', 'MEXICAN_ER', 'NOK_BID',
       'NOK_OFFER', 'NOK_ER', 'NORWEGIAN_BID', 'NORWEGIAN_OFFER',
       'NORWEGIAN_ER', 'PLN_BID', 'PLN_OFFER', 'PLN_ER', 'POLISH_BID',
       'POLISH_OFFER', 'POLISH_ER', 'RUSSIAN_BID', 'RUSSIAN_OFFER',
       'RUSSIAN_ER', 'SEK_BID', 'SEK_OFFER', 'SEK_ER', 'SINGAPORE_BID',
       'SINGAPORE_OFFER', 'SINGAPORE_ER', 'SOUTH_BID', 'SOUTH_OFFE

In [44]:
usd_df.columns

Index(['AUSTRALIAN_BID', 'AUSTRALIAN_ER', 'AUSTRALIAN_OFFER', 'BRAZILIAN_BID',
       'BRAZILIAN_ER', 'BRAZILIAN_OFFER', 'BULGARIAN_BID', 'BULGARIAN_ER',
       'BULGARIAN_OFFER', 'CANADIAN_BID', 'CANADIAN_ER', 'CANADIAN_OFFER',
       'CHILEAN_BID', 'CHILEAN_ER', 'CHILEAN_OFFER', 'CROATIAN_BID',
       'CROATIAN_ER', 'CROATIAN_OFFER', 'CZECH_BID', 'CZECH_ER', 'CZECH_OFFER',
       'EURO_BID', 'EURO_ER', 'EURO_OFFER', 'HUNGARIAN_BID', 'HUNGARIAN_ER',
       'HUNGARIAN_OFFER', 'INDIAN_BID', 'INDIAN_ER', 'INDIAN_OFFER',
       'INDONESIAN_BID', 'INDONESIAN_ER', 'INDONESIAN_OFFER', 'ISRAELI_BID',
       'ISRAELI_ER', 'ISRAELI_OFFER', 'JAPANESE_BID', 'JAPANESE_ER',
       'JAPANESE_OFFER', 'MEXICAN_BID', 'MEXICAN_ER', 'MEXICAN_OFFER',
       'NEW_BID', 'NEW_ER', 'NEW_OFFER', 'NORWEGIAN_BID', 'NORWEGIAN_ER',
       'NORWEGIAN_OFFER', 'PHILIPPINE_BID', 'PHILIPPINE_ER',
       'PHILIPPINE_OFFER', 'POLISH_BID', 'POLISH_ER', 'POLISH_OFFER',
       'RUSSIAN_BID', 'RUSSIAN_ER', 'RUSSIAN_OFFER', '